<a href="https://colab.research.google.com/github/HalaAl-Omary/Capstone-Project-_Interactive-Dashboard/blob/main/HalaALOmary_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Capstone Project - Interactive Dashboard

###Step 1: Install Libraries and Import Dependencies

In [ ]:
!pip install streamlit -q
!pip install pyngrok -q

import streamlit as st
import pandas as pd
import plotly.express as px
from pyngrok import ngrok
import os
import subprocess
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.2 MB/s eta 0:00:00


###Step 2: Load and Prepare the Data

In [ ]:
url ="https://raw.githubusercontent.com/Steven-Alvarado/Global-Temperature-Analysis/refs/heads/main/GlobalTemperatures.csv"
df = pd.read_csv(url)
# df.head()
# df.info()
df['dt'] = pd.to_datetime(df['dt'])
df['Year'] = df['dt'].dt.year
# df['Year']
# df['LandAverageTemperature'].info()
df.dropna(subset=['LandAverageTemperature'], inplace=True)
df

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty,Year
0,1850-01-01,0.749,1.105,8.242,1.738,-3.206,2.822,12.833,0.367,1850
1,1850-02-01,3.071,1.275,9.970,3.007,-2.291,1.623,13.588,0.414,1850
2,1850-03-01,4.954,0.955,10.347,2.401,-1.905,1.410,14.043,0.341,1850
3,1850-04-01,7.217,0.665,12.934,1.004,1.018,1.329,14.667,0.267,1850
4,1850-05-01,10.004,0.617,15.655,2.406,3.811,1.347,15.507,0.249,1850
...,...,...,...,...,...,...,...,...,...,...
1987,2015-08-01,14.755,0.072,20.699,0.110,9.005,0.170,17.589,0.057,2015
1988,2015-09-01,12.999,0.079,18.845,0.088,7.199,0.229,17.049,0.058,2015
1989,2015-10-01,10.801,0.102,16.450,0.059,5.232,0.115,16.290,0.062,2015
1990,2015-11-01,7.433,0.119,12.892,0.093,2.157,0.106,15.252,0.063,2015


###Step 3: Build the Streamlit App

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
st.set_page_config(
    page_title="Global Temperature Dashboard",
    page_icon="🌍",
    layout="wide"
)
@st.cache_data
def load_data():
    url = "https://raw.githubusercontent.com/Steven-Alvarado/Global-Temperature-Analysis/refs/heads/main/GlobalTemperatures.csv"
    df = pd.read_csv(url)
    df['dt'] = pd.to_datetime(df['dt'])
    df['Year'] = df['dt'].dt.year
    df.dropna(subset=['LandAverageTemperature'], inplace=True)
    return df

df = load_data()
st.title("Global Temperature Trends Dashboard")
st.markdown("Explore how average land temperatures have changed over time.")
st.sidebar.header("Filters")
year_range = st.sidebar.slider("Select Year Range",
                              int(df['Year'].min()),
                              int(df['Year'].max()),
                              (1900,2015)
                              )
filtered_df = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1])]
st.write(f"## Temperature Trend from {year_range[0]} to {year_range[1]}")
yearly_temp = filtered_df.groupby('Year')['LandAverageTemperature'].mean().reset_index()
st.write("### Average Temperature Trend")
fig_line = px.line(
    yearly_temp,
    x='Year',
    y='LandAverageTemperature',
    title='Global Average Land Temperature Over Time',
    labels={'Year': 'Year', 'LandAverageTemperature': 'Average Temperature (°C)'}
)
fig_line.update_traces(mode='lines+markers')
fig_line.add_scatter(x=yearly_temp['Year'], y=yearly_temp['LandAverageTemperature'].rolling(window=10).mean(), mode='lines', name='10-Year Moving Average')
st.plotly_chart(fig_line, use_container_width=True)
st.write("### Raw Data")
with st.expander("Show Filtered Data"):
    st.dataframe(filtered_df)


Writing app.py


In [ ]:
authtoken = "33NNp4yunF3sa59CtjWFeS7oiO0_2Fk22thZL69xzVupzp1YZ"

from IPython.display import display, IFrame
import subprocess
import time
from pyngrok import ngrok

if authtoken:
    try:
        ngrok.kill()
        ngrok.set_auth_token(authtoken)
        process = subprocess.Popen(["streamlit", "run", "app.py"])
        print("Starting Streamlit server in the background...")
        time.sleep(5)
        public_url = ngrok.connect(addr='8501')
        print(f"Streamlit App URL: {public_url}")
        display(IFrame(public_url, width='100%', height=800))
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your authtoken is correct and that Streamlit is installed.")
else:
    print("Please enter your ngrok authtoken in the field above to run the app.")


Starting Streamlit server in the background...
Streamlit App URL: NgrokTunnel: "https://woodrow-clustery-anagrammatically.ngrok-free.dev" -> "http://localhost:8501"
